In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import time
plt.ion()

class DataFunction:
    def __init__(self,dim=2):
        self.dim = dim
    @property
    def ninput(self): return self.dim
    @property
    def noutput(self): return 1
    def eval(self,x):
        # Return the sum of square while taking care of the dimension
        return np.array([np.sum(x**2,axis=x.ndim-1)]).T
    def __call__(self,x): return self.eval(x)
    
datagen = DataFunction()

NSAMPLE = 1000
NINPUT = datagen.ninput
NOUTPUT = datagen.noutput
NHIDDEN = 64

# Create random Tensors to hold inputs and outputs
x = np.random.rand(NSAMPLE, NINPUT)*4-2
y = datagen(x)

x = torch.Tensor(x)
y = torch.Tensor(y)


class Model(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super().__init__()
        self.l1 = torch.nn.Linear(D_in, H)
        self.l2 = torch.nn.Linear(H, H)
        self.l3 =torch.nn.Linear(H, D_out)

        self.activ1 = torch.tanh#torch.nn.ReLU()
        self.activ2 = torch.tanh#torch.nn.ReLU()

    def forward(self, X):
        return self.l3(self.activ2(self.l2(self.activ1(self.l1(X)))))

########### Training


model = Model(NINPUT,NHIDDEN,NOUTPUT)

loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
t0 = time.time()
for t in range(NSAMPLE*20):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if not t % 1000: print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

print('Training lasted = %.0f seconds' % (time.time()-t0))

############  Test/validation
"""

plt.figure(1)
y_last = model(x)
plt.plot(y.detach().numpy(), y_last.detach().numpy(), 'o')
plt.xlabel('Targeted y', fontsize=16)
plt.ylabel('Modeled y', fontsize=16)
plt.savefig('Model_validation')

if NINPUT == 2:
    xrange = np.linspace(-1,1,100)
    xtest = np.array([ [x1,x2] for x1 in xrange for x2 in xrange ])
    ytest = datagen(xtest)
    ypred = model(torch.Tensor(xtest)).detach().numpy()
    plt.figure(2,figsize=[10,15])

    trange = [ min(ytest),max(ytest) ]
    prange = [ min(ypred),max(ypred) ]
    vrange = [ min(trange+prange),max(trange+prange)]
    vd  = vrange[1]-vrange[0]
    vrange = [ vrange[0]-vd*.1, vrange[1]+vd*.1 ]
    
    plt.subplot(3,1,1)
    plt.scatter(xtest[:,0],xtest[:,1],c=ytest.flat,vmin=vrange[0],vmax=vrange[1])
    plt.colorbar()
    plt.title('Data')

    plt.subplot(3,1,2)
    plt.scatter(xtest[:,0],xtest[:,1],c=ypred.flat,vmin=vrange[0],vmax=vrange[1])
    plt.colorbar()
    plt.title('Prediction')

    plt.subplot(3,1,3)
    plt.scatter(xtest[:,0],xtest[:,1],c=(ypred-ytest).flat)
    plt.colorbar()
    plt.title('Error')
"""    
    

0 10872.2958984375
1000 3366.804443359375
2000 524.8036499023438
3000 70.91944122314453
4000 27.638893127441406
5000 10.332592010498047
6000 4.092840671539307
7000 1.5842965841293335
8000 0.7119300365447998
9000 0.4461432695388794
10000 0.29155704379081726
11000 0.1746618151664734
12000 0.09589801728725433
13000 0.05314129218459129
14000 0.03150360658764839
15000 0.020917922258377075
16000 0.015380112454295158
17000 0.012230257503688335
18000 0.010033935308456421
19000 0.008401062339544296
Training lasted = 32 seconds


"\n\nplt.figure(1)\ny_last = model(x)\nplt.plot(y.detach().numpy(), y_last.detach().numpy(), 'o')\nplt.xlabel('Targeted y', fontsize=16)\nplt.ylabel('Modeled y', fontsize=16)\nplt.savefig('Model_validation')\n\nif NINPUT == 2:\n    xrange = np.linspace(-1,1,100)\n    xtest = np.array([ [x1,x2] for x1 in xrange for x2 in xrange ])\n    ytest = datagen(xtest)\n    ypred = model(torch.Tensor(xtest)).detach().numpy()\n    plt.figure(2,figsize=[10,15])\n\n    trange = [ min(ytest),max(ytest) ]\n    prange = [ min(ypred),max(ypred) ]\n    vrange = [ min(trange+prange),max(trange+prange)]\n    vd  = vrange[1]-vrange[0]\n    vrange = [ vrange[0]-vd*.1, vrange[1]+vd*.1 ]\n    \n    plt.subplot(3,1,1)\n    plt.scatter(xtest[:,0],xtest[:,1],c=ytest.flat,vmin=vrange[0],vmax=vrange[1])\n    plt.colorbar()\n    plt.title('Data')\n\n    plt.subplot(3,1,2)\n    plt.scatter(xtest[:,0],xtest[:,1],c=ypred.flat,vmin=vrange[0],vmax=vrange[1])\n    plt.colorbar()\n    plt.title('Prediction')\n\n    plt.subp

In [3]:
x = np.random.rand(NSAMPLE, NINPUT)*4-2
y = datagen(x)


In [7]:
NSAMPLE * 20

20000